# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import gmaps.datasets

from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [16]:
weather_csv_file = 'weather_data.csv'
weather_df = pd.read_csv(weather_csv_file)

weather_df.head()

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Caxito,75,AO,1612480190,78,-8.5785,13.6643,80.60,6.91
1,1,Tuktoyaktuk,75,CA,1612480134,67,69.4541,-133.0374,-34.60,11.50
2,2,Bredasdorp,90,ZA,1612480190,77,-34.5322,20.0403,68.00,5.75
3,3,Mar del Plata,40,AR,1612480186,77,-38.0023,-57.5575,64.99,10.36
4,4,Atuona,25,PF,1612480190,71,-9.8000,-139.0333,78.58,10.98


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:

# Store 'Lat' and 'Lng' into  locations 
locations = weather_df[["Lat", "Lng"]].astype(float)

#weatherdf = weatherdf.dropna()

hum_rate = weather_df["Humidity"].astype(float)


# Create a poverty Heatmap layer
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=hum_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)

fig.add_layer(heat_layer)

plt.savefig("vacationheatmap.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
#Narrow down the DataFrame to find your ideal weather condition.

#A max temperature lower than 80 degrees but higher than 70.
first = weather_df.loc[weather_df["Max Temp"]<80,:]
second = first.loc[weather_df["Max Temp"]>70,:]
second.head()

#Wind speed less than 10 mph.
third = second.loc[second["Wind Speed"]<10,:]
third

#Zero cloudiness.
fourth = third.loc[third["Cloudiness"]==0,:]
fourth.head()

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
128,128,Jizan,0,SA,1612480208,73,17.3333,42.6667,75.20,8.05
165,165,Wamba,0,NG,1612480214,17,8.9333,8.6000,77.34,5.68
175,175,Boundiali,0,CI,1612480216,20,9.5217,-6.4869,74.84,2.21
300,300,Mongo,0,TD,1612480236,22,12.1844,18.6930,70.27,9.57
355,355,Conde,0,BR,1612480252,83,-7.2597,-34.9075,78.80,6.91


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = pd.DataFrame(weather_df, columns=["City", "Country", "Lat", "Lng"])

hotel_df["Hotel Name"] = ""
# hotel_df["Coordinates"]=""
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Caxito,AO,-8.5785,13.6643,
1,Tuktoyaktuk,CA,69.4541,-133.0374,
2,Bredasdorp,ZA,-34.5322,20.0403,
3,Mar del Plata,AR,-38.0023,-57.5575,
4,Atuona,PF,-9.8000,-139.0333,
...,...,...,...,...,...
549,Santa Maria,BR,-29.6842,-53.8069,
550,Cadillac,US,44.2519,-85.4012,
551,Tucumã,BR,-0.1833,-50.0500,
552,Guasdualito,VE,7.2424,-70.7324,


In [19]:
# Set parameters to search for a hotel
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": "1cd9eccbef34a95d7aa66e8954eafd31"
}

# Iterate through 
# Use the lat/lng to identify hotels
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude 
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_

{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_

{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_

{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_

{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_

...

{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_

{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_

{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_

{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_

,City,Country,Lat,Lng,Hotel Name
0,Caxito,AO,-8.5785,13.6643,
1,Tuktoyaktuk,CA,69.4541,-133.0374,
2,Bredasdorp,ZA,-34.5322,20.0403,
3,Mar del Plata,AR,-38.0023,-57.5575,
4,Atuona,PF,-9.8000,-139.0333,
...,...,...,...,...,...
549,Santa Maria,BR,-29.6842,-53.8069,
550,Cadillac,US,44.2519,-85.4012,
551,Tucumã,BR,-0.1833,-50.0500,
552,Guasdualito,VE,7.2424,-70.7324,


In [20]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)
# Display figure
fig